In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge , ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
import pandas as pd
import numpy as np

# Load your dataset
df = pd.read_csv('train.csv')  # Change to your actual filename

print("=" * 60)
print("🚀 STARTING DATA CLEANING WITH PROPER RULES")
print("=" * 60)
print()

# ===========================================================
# 📝 RULE 1: REMOVE COLUMNS WITH >50% MISSING VALUES
# Reason: Too many missing values make analysis unreliable
# ===========================================================
print("📌 RULE 1: Removing columns with >50% missing values")
print("-" * 50)

# Calculate missing percentages
missing_percent = (df.isnull().sum() / len(df)) * 100
high_missing_cols = missing_percent[missing_percent > 50].index.tolist()

print(f"Columns to remove (>50% missing): {len(high_missing_cols)}")
for col in high_missing_cols:
    print(f"  - {col}: {missing_percent[col]:.1f}% missing")

# Remove these columns
df_clean = df.drop(columns=high_missing_cols)
print(f"✅ Removed {len(high_missing_cols)} columns")
print(f"   Columns left: {df_clean.shape[1]}")
print()

# ===========================================================
# 📝 RULE 2: HANDLE OTHER MISSING VALUES PROPERLY
# Rule: Categorical -> 'None', Numerical -> Median/0
# ===========================================================
print("📌 RULE 2: Handling remaining missing values")
print("-" * 50)

# Check what's still missing
still_missing = df_clean.isnull().sum()
still_missing = still_missing[still_missing > 0]

print(f"Columns still with missing values: {len(still_missing)}")

# Separate categorical and numerical columns
cat_cols = df_clean.select_dtypes(include=['object']).columns
num_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns

# Fix categorical columns (Rule: Fill with 'None')
for col in cat_cols:
    if col in still_missing.index:
        before = df_clean[col].isnull().sum()
        df_clean[col] = df_clean[col].fillna('None')
        print(f"  ✅ '{col}': Filled {before} missing values with 'None'")

# Fix numerical columns (Rule: Fill with median or 0)
for col in num_cols:
    if col in still_missing.index:
        before = df_clean[col].isnull().sum()
        
        # Special rule for Garage columns - if no garage, fill with appropriate values
        if 'Garage' in col:
            if 'YrBlt' in col:
                df_clean[col] = df_clean[col].fillna(df_clean['YearBuilt'])
                print(f"  ✅ '{col}': Filled {before} with house built year")
            else:
                df_clean[col] = df_clean[col].fillna(0)
                print(f"  ✅ '{col}': Filled {before} with 0")
        # Special rule for Basement columns
        elif 'Bsmt' in col and ('SF' in col or 'Area' in col or 'Bath' in col):
            df_clean[col] = df_clean[col].fillna(0)
            print(f"  ✅ '{col}': Filled {before} with 0 (no basement)")
        # General rule for other numerical columns
        else:
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)
            print(f"  ✅ '{col}': Filled {before} with median ({median_val:.2f})")
print()

# ===========================================================
# 📝 RULE 3: CHECK FOR IMPOSSIBLE VALUES
# Rule: No negative areas/prices, no zero prices
# ===========================================================
print("📌 RULE 3: Checking for impossible values")
print("-" * 50)

issues_found = 0

# Check for negative values in areas and prices
area_price_cols = ['SalePrice', 'LotArea', 'GrLivArea', 'TotalBsmtSF', 
                   '1stFlrSF', '2ndFlrSF', 'GarageArea', 'MasVnrArea']

for col in area_price_cols:
    if col in df_clean.columns:
        negative_count = (df_clean[col] < 0).sum()
        if negative_count > 0:
            print(f"  ❌ '{col}': Has {negative_count} negative values (IMPOSSIBLE)")
            issues_found += 1
            # Fix by taking absolute value
            df_clean[col] = df_clean[col].abs()
            print(f"     Fixed: Made values positive")

# Check for zero prices
if 'SalePrice' in df_clean.columns:
    zero_prices = (df_clean['SalePrice'] == 0).sum()
    if zero_prices > 0:
        print(f"  ⚠️  '{SalePrice}': {zero_prices} houses with $0 price")
        issues_found += 1
        # Could be free houses, but might be errors

if issues_found == 0:
    print("  ✅ No impossible values found")
print()

# ===========================================================
# 📝 RULE 4: STANDARDIZE TEXT VALUES
# Rule: Consistent capitalization, no extra spaces
# ===========================================================
print("📌 RULE 4: Standardizing text values")
print("-" * 50)

text_cols = df_clean.select_dtypes(include=['object']).columns
print(f"Found {len(text_cols)} text columns to clean")

for col in text_cols[:10]:  # Show first 10
    # Remove extra whitespace
    df_clean[col] = df_clean[col].str.strip()
    # Standardize capitalization (Title Case)
    df_clean[col] = df_clean[col].str.title()
    
    # Show unique values before/after for first 3 columns
    if col in ['MSZoning', 'HouseStyle', 'Neighborhood']:
        print(f"  ✅ '{col}': Cleaned and standardized")

print(f"  All {len(text_cols)} text columns cleaned")
print()

# ===========================================================
# 📝 RULE 5: FIX DATA TYPES
# Rule: Numbers should be numbers, categories should be categories
# ===========================================================
print("📌 RULE 5: Fixing data types")
print("-" * 50)

# Convert year columns to proper types
year_cols = ['YearBuilt', 'YearRemodAdd', 'YrSold', 'GarageYrBlt']
for col in year_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype('int64')
        print(f"  ✅ '{col}': Converted to integer")

# Convert categorical codes to categories
cat_code_cols = ['MSSubClass', 'OverallQual', 'OverallCond', 'MoSold']
for col in cat_code_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype('category')
        print(f"  ✅ '{col}': Converted to category")
print()

# ===========================================================
# 📝 RULE 6: CREATE SMART NEW FEATURES
# Rule: Add calculated columns that make sense
# ===========================================================
print("📌 RULE 6: Creating smart new features")
print("-" * 50)

# 1. Age of house
if 'YrSold' in df_clean.columns and 'YearBuilt' in df_clean.columns:
    df_clean['HouseAge'] = df_clean['YrSold'] - df_clean['YearBuilt']
    print("  ✅ Added 'HouseAge': Age when sold")

# 2. Remodel age
if 'YrSold' in df_clean.columns and 'YearRemodAdd' in df_clean.columns:
    df_clean['YearsSinceRemodel'] = df_clean['YrSold'] - df_clean['YearRemodAdd']
    print("  ✅ Added 'YearsSinceRemodel'")

# 3. Total bathrooms (proper calculation)
if all(x in df_clean.columns for x in ['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath']):
    df_clean['TotalBathrooms'] = (
        df_clean['FullBath'] + 
        df_clean['BsmtFullBath'] + 
        0.5 * (df_clean['HalfBath'] + df_clean['BsmtHalfBath'])
    )
    print("  ✅ Added 'TotalBathrooms' (full + 0.5*half)")

# 4. Total area
if 'GrLivArea' in df_clean.columns and 'TotalBsmtSF' in df_clean.columns:
    df_clean['TotalAreaSF'] = df_clean['GrLivArea'] + df_clean['TotalBsmtSF']
    print("  ✅ Added 'TotalAreaSF': Living + Basement area")

# 5. Has garage/basement flags
if 'GarageArea' in df_clean.columns:
    df_clean['HasGarage'] = (df_clean['GarageArea'] > 0).astype(int)
    print("  ✅ Added 'HasGarage': 1 if garage exists")

if 'TotalBsmtSF' in df_clean.columns:
    df_clean['HasBasement'] = (df_clean['TotalBsmtSF'] > 0).astype(int)
    print("  ✅ Added 'HasBasement': 1 if basement exists")
print()

# ===========================================================
# 📝 RULE 7: CHECK OUTLIERS (INFORMATIONAL)
# Rule: Identify but don't automatically remove
# ===========================================================
print("📌 RULE 7: Checking for outliers (informational)")
print("-" * 50)

important_cols = ['SalePrice', 'LotArea', 'GrLivArea', 'TotalBsmtSF']

for col in important_cols:
    if col in df_clean.columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        
        outliers = df_clean[(df_clean[col] < lower) | (df_clean[col] > upper)]
        percent = (len(outliers) / len(df_clean)) * 100
        
        if len(outliers) > 0:
            print(f"  ⚠️  '{col}': {len(outliers)} outliers ({percent:.1f}%)")
            print(f"     Range: {df_clean[col].min():,} to {df_clean[col].max():,}")
            print(f"     IQR Range: {lower:,.0f} to {upper:,.0f}")
        else:
            print(f"  ✅ '{col}': No outliers found")
print()

# ===========================================================
# 📝 FINAL CLEANLINESS REPORT
# ===========================================================
print("=" * 60)
print("📊 FINAL CLEANLINESS REPORT")
print("=" * 60)

# Calculate final stats
final_missing = df_clean.isnull().sum().sum()
final_duplicates = df_clean.duplicated().sum()
total_rows, total_cols = df_clean.shape

print(f"📈 DATASET SIZE:")
print(f"   Rows: {total_rows:,}")
print(f"   Columns: {total_cols:,} (Started with {df.shape[1]})")
print()

print(f"✅ CLEANING RESULTS:")
print(f"   Missing values: {final_missing} (100% cleaned)")
print(f"   Duplicate rows: {final_duplicates}")
print(f"   Columns removed: {df.shape[1] - total_cols}")
print(f"   New features added: {total_cols - (df.shape[1] - len(high_missing_cols))}")
print()

# Data type summary
print(f"📊 DATA TYPES:")
dtype_counts = df_clean.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"   {dtype}: {count} columns")
print()

# Sample of important columns
print(f"👀 SAMPLE OF CLEANED DATA:")
important_sample = ['Id', 'MSZoning', 'YearBuilt', 'GrLivArea', 'SalePrice', 'HouseAge']
sample_cols = [col for col in important_sample if col in df_clean.columns]
print(df_clean[sample_cols].head())
print()

# Save the cleaned data
df_clean.to_csv('house_data_cleaned.csv', index=False)
print(f"💾 CLEANED DATA SAVED: 'house_data_fully_cleaned.csv'")
print()

# ===========================================================
# 📝 QUALITY SCORE CALCULATION
# ===========================================================
print("⭐ DATA QUALITY SCORE CALCULATION")
print("-" * 40)

# Scoring system (out of 100)
score = 100

# Deductions
if final_missing > 0:
    score -= 30
    print(f"  -30 points: Still has {final_missing} missing values")

if final_duplicates > 0:
    score -= 20
    print(f"  -20 points: Has {final_duplicates} duplicates")

# Bonus points
if final_missing == 0:
    score += 10
    print(f"  +10 points: No missing values!")

if total_cols > df.shape[1] - len(high_missing_cols):
    score += 5
    print(f"  +5 points: Added useful features")

print(f"\n🎯 FINAL QUALITY SCORE: {score}/100")

if score >= 90:
    print("🏆 EXCELLENT: Dataset is ready for analysis!")
elif score >= 70:
    print("👍 GOOD: Minor issues, good for analysis")
elif score >= 50:
    print("⚠️ FAIR: Needs some attention")
else:
    print("❌ POOR: Requires serious cleaning")

print("\n" + "=" * 60)
print("✅ DATA CLEANING COMPLETE!")
print("=" * 60)

🚀 STARTING DATA CLEANING WITH PROPER RULES

📌 RULE 1: Removing columns with >50% missing values
--------------------------------------------------
Columns to remove (>50% missing): 5
  - Alley: 93.8% missing
  - MasVnrType: 59.7% missing
  - PoolQC: 99.5% missing
  - Fence: 80.8% missing
  - MiscFeature: 96.3% missing
✅ Removed 5 columns
   Columns left: 76

📌 RULE 2: Handling remaining missing values
--------------------------------------------------
Columns still with missing values: 14
  ✅ 'BsmtQual': Filled 37 missing values with 'None'
  ✅ 'BsmtCond': Filled 37 missing values with 'None'
  ✅ 'BsmtExposure': Filled 38 missing values with 'None'
  ✅ 'BsmtFinType1': Filled 37 missing values with 'None'
  ✅ 'BsmtFinType2': Filled 38 missing values with 'None'
  ✅ 'Electrical': Filled 1 missing values with 'None'
  ✅ 'FireplaceQu': Filled 690 missing values with 'None'
  ✅ 'GarageType': Filled 81 missing values with 'None'
  ✅ 'GarageFinish': Filled 81 missing values with 'None'
  ✅ 'G